In [23]:
import arcpy
## stop outputs being added to the map
arcpy.env.addOutputsToMap = False

#### dissolve by GOV_TYPE
wdpa24 = 'O:/PP2024/test.gdb/wdpa_gov_type_test'
wdpa22 = 'O:/PP2024/test.gdb/wdpa_gov_type_test_1'

datasets = [wdpa24, wdpa22]

outputs = ['O:/PP2024/test_output.gdb/wdpa_gov_type_test_diss',
          'O:/PP2024/test_output.gdb/wdpa_gov_type_test_1_diss'
         ]

for dataset, output in zip(datasets, outputs):
    arcpy.analysis.PairwiseDissolve(
        in_features = dataset, 
        out_feature_class = output, 
        dissolve_field = "GOV_TYPE", 
        multi_part = "SINGLE_PART"
    )

for i in outputs:
    arcpy.management.RepairGeometry(i, "DELETE_NULL", "OGC")
    print("repair complete")

repair complete
repair complete


In [1]:
### union with itself to identify overlapping GOV TYPES
datasets = ['O:/PP2024/gov_type.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE',
            'O:/PP2024/gov_type.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE',
            'O:/PP2024/gov_type.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE',
            'O:/PP2024/gov_type.gdb/wdoecm_poly_point_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE',
            'O:/PP2024/gov_type.gdb/wdoecm_poly_point_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE',
            'O:/PP2024/gov_type.gdb/wdoecm_poly_point_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE'
           ]

outputs = ['O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union',
            'O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE_union',
            'O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE_union',
            'O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union',
            'O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE_union',
            'O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE_union'
           ]

for dataset, output in zip(datasets, outputs):
    arcpy.analysis.Union(
        in_features = dataset, 
        out_feature_class = output
    )

for i in outputs:
    arcpy.management.RepairGeometry(i, "DELETE_NULL", "OGC")
    print("repair complete")

repair complete
repair complete
repair complete
repair complete
repair complete
repair complete


In [2]:
## add centroids
for i in outputs:
     arcpy.management.CalculateGeometryAttributes(
        in_features = i,
        geometry_property = [["CENTROID_X", "CENTROID_X"], ["CENTROID_Y", "CENTROID_Y"]]
     )


In [3]:
### add new field and sum x y centroids
# Loop through each feature class
for i in outputs:
    # Add a new field for storing the sum of length and area
    arcpy.AddField_management(i, "XY", "FLOAT")

    # Calculate the sum of the length and area fields for each feature
    with arcpy.da.UpdateCursor(i, ["CENTROID_X", "CENTROID_Y", "XY"]) as cursor:
        for row in cursor:
            # Calculate the sum
            row[2] = row[0] + row[1]
            # Update the row
            cursor.updateRow(row)

In [4]:
### where XY value is duplicated, change gov type to Mixed
# Loop through each feature class
for fc in outputs:
    print(f"Processing {fc}...")

    # Dictionary to keep track of the count of each XY value
    xy_dict = {}

    # First pass: count occurrences of each XY value
    with arcpy.da.SearchCursor(fc, ["XY"]) as search_cursor:
        for row in search_cursor:
            xy_value = row[0]
            if xy_value in xy_dict:
                xy_dict[xy_value] += 1
            else:
                xy_dict[xy_value] = 1

    # Second pass: update GOV_TYPE where XY values are duplicated
    with arcpy.da.UpdateCursor(fc, ["XY", "GOV_TYPE"]) as update_cursor:
        for row in update_cursor:
            xy_value = row[0]
            if xy_dict[xy_value] > 1:  # If XY value is duplicated
                row[1] = "Mixed"
                update_cursor.updateRow(row)

    print(f"Finished updating {fc}.")

print("Completed processing all feature classes.")

Processing O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union...
Finished updating O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union.
Processing O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE_union...
Finished updating O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2022_Aug2024_GOV_TYPE_union.
Processing O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE_union...
Finished updating O:/PP2024/gov_type_flat.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2020_Aug2024_GOV_TYPE_union.
Processing O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2024_Aug2024_GOV_TYPE_union...
Finished updating O:/PP2024/gov_type_flat.gdb/wdoecm_poly_point_basemap_intersect_statusyr2

In [5]:
### remove overlaps with identical XY value - this will leave a flat layer
for i in outputs:
    arcpy.management.DeleteIdentical(i, "XY")